In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('User Raw to Trusted') \
    .config('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension') \
    .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.delta.catalog.DeltaCatalog') \
    .config("spark.sql.warehouse.dir", "/opt/hive/data/warehouse") \
    .enableHiveSupport()\
    .getOrCreate()

In [5]:
posicao_veiculo = 's3a://raw/posicao_veiculo'

In [6]:
# df_parada = spark.read.json(parada)
df_posicao_veiculo = spark.read.json(posicao_veiculo)

In [7]:
df_posicao_veiculo.printSchema()

root
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



In [6]:
from pyspark.sql.functions import explode, col
df_posicao_veiculo2 = df_posicao_veiculo.withColumn("dict", explode(col('l')))

In [ ]:
df_posicao_veiculo2.select("dict").show(truncate=False)

In [52]:
df_posicao_veiculo2.printSchema()

root
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)
 |-- dict: struct (nullable = true)
 |    |-- c: string (nullable = true)
 |    |-- cl: long (nullable = true)
 |    |-- lt0: string (nullable = t

In [7]:
df_posicao_veiculo3 = df_posicao_veiculo2.select(
    col("dict.c").alias("LETREIRO_LINHA"), 
    col("dict.cl").alias("ID_LINHA"), 
    col("dict.lt0").alias("LETREIRO_DESTINO_LINHA"), 
    col("dict.lt1").alias("LETREIRO_ORIGEM_LINHA"), 
    col("dict.qv").alias("QTD_VEICULO_LINHA"), 
    col("dict.sl").alias("SENTIDO_O_LINHA"), 
    explode("dict.vs").alias("veiculos"),
    col("veiculos.a").alias("ACESSIVEL_VEICULO"),
    col("veiculos.p").alias("ID_VEICULO"),
    col("veiculos.px").alias("LONGITUDE_VEICULO"),
    col("veiculos.py").alias("LATITUDE_VEICULO")
).drop("veiculos")

In [ ]:
df_posicao_veiculo.show()

In [8]:
silver_path_posicao_veiculo = 's3a://trusted/posicao_veiculo'

In [9]:
df_posicao_veiculo.write.format('delta')\
    .mode('overwrite').option("overwriteSchema", "true").save(silver_path_posicao_veiculo)

NameError: name 'df_posicao_veiculo' is not defined

In [10]:
df_posicao_to_previsao = spark.read.format('delta') \
  .load(silver_path_posicao_veiculo)

In [12]:
spark.sql("SHOW DATABASES").show()

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [11]:
spark.sql("CREATE DATABASE SPTRANS")

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [ ]:
df_posicao_to_previsao.saveAsTable("SPTRANS.POSICAO_VEICULO")

In [59]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, lit
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType


In [ ]:
http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=d3f026bb74699c88e75ef5a7e71cf1181e70cd26cb93b8b31038c9df15ba2f61

In [ ]:
import requests
from pyspark.sql import Row
import json
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, BooleanType, TimestampType, FloatType

url = "http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=d3f026bb74699c88e75ef5a7e71cf1181e70cd26cb93b8b31038c9df15ba2f61"
response = requests.post(url)

if response.status_code == 200:
  cookies = response.cookies
  # Assuming you want to fetch data from another endpoint using the authentication cookie
  api_endpoint = "http://api.olhovivo.sptrans.com.br/v2.1/posicao"  # Replace with the desired endpoint
  response_api = requests.get(api_endpoint, cookies=cookies)

  if response_api.status_code == 200:
    data = response_api.json()  # Convert JSON to a list of Rows
    print(data)
    df = spark.createDataFrame(data, schema=schema)
    df.show()
  else:
    print(f"Error fetching data from API: {response_api.status_code}")
else:
  print(f"Error authenticating: {response.status_code}")

In [59]:
import requests
from pyspark.sql.functions import udf
from http.cookiejar import LWPCookieJar
import datetime
import time

post_auth = "http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=d3f026bb74699c88e75ef5a7e71cf1181e70cd26cb93b8b31038c9df15ba2f61"
session = requests.Session()
session.post(post_auth)

def fetch_data(id_linha):
    url = "http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=d3f026bb74699c88e75ef5a7e71cf1181e70cd26cb93b8b31038c9df15ba2f61"
    session = requests.Session()
    response = session.post(url)
    
    if response.status_code == 200:
        cookies = response.cookies
        
        endpoint = "http://api.olhovivo.sptrans.com.br/v2.1/Previsao/Linha?"
        params = {
            "codigoLinha": id_linha,
        }
        response = session.get(endpoint, params=params, cookies=cookies)

        time = datetime.datetime.now()
        bucket = "raw"
        name_file = f"previsao_chegada_{time}.json"
        caminho_de_salvar = f"/previsao_chegada/{name_file}"

        upload_json_to_minio(
            data=response.json(), 
            bucket=bucket, 
            object_name=name_file, 
            path_save=caminho_de_salvar
        )
        return True  # assuming API returns a list of records
    else:
        print("NAODEUCERTO")

convertUDF = udf(lambda id: fetch_data(id))

df_previsao_responde = df_posicao_to_previsao.withColumn("response", convertUDF(col("id_linha"))).select("response")

In [60]:
df_previsao_responde.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [46]:
import json
import boto3
from botocore.client import Config
from botocore.exceptions import NoCredentialsError
from io import BytesIO

def upload_json_to_minio(data, bucket, object_name, path_save):
    # Configurar o cliente boto3 para MinIO
    s3_client = boto3.client(
        's3',
        endpoint_url='http://minio:9000',  # Altere para o endpoint do seu MinIO
        aws_access_key_id='datalake',        # Altere para sua chave de acesso do MinIO
        aws_secret_access_key='datalake',    # Altere para sua chave secreta do MinIO
    )

    try:
        print("Serializar o conteúdo JSON e convertê-lo para bytes")
        json_bytes = json.dumps(data).encode('utf-8')

        print("Criar um buffer de bytes para simular um arquivo")
        json_buffer = BytesIO(json_bytes)

        # Upload do "arquivo" JSON para o MinIO
        s3_client.upload_fileobj(
            json_buffer,
            bucket,
            path_save
         )
        print(f'Arquivo JSON foi carregado com sucesso no bucket {bucket}, objeto: {object_name}')
    
    except NoCredentialsError:
        print("Credenciais não encontradas")

In [62]:
raw_path_posicao_veiculo = 's3a://raw/previsao_chegada'

In [63]:
df_posicao_veiculo = spark.read.json(raw_path_posicao_veiculo)

In [65]:
df_posicao_to_previsao.printSchema()

root
 |-- hr: string (nullable = true)
 |-- ps: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cp: long (nullable = true)
 |    |    |-- np: string (nullable = true)
 |    |    |-- px: double (nullable = true)
 |    |    |-- py: double (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: string (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- t: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



In [87]:
df_explode = df_posicao_to_previsao.withColumn("ps_explod", explode("ps"))\
    .withColumn("vs_explod", explode("ps_explod.vs")).select("ps_explod","vs_explod")

In [89]:
df_explode.printSchema()

root
 |-- ps_explod: struct (nullable = true)
 |    |-- cp: long (nullable = true)
 |    |-- np: string (nullable = true)
 |    |-- px: double (nullable = true)
 |    |-- py: double (nullable = true)
 |    |-- vs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |-- is: string (nullable = true)
 |    |    |    |-- p: string (nullable = true)
 |    |    |    |-- px: double (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- sv: string (nullable = true)
 |    |    |    |-- t: string (nullable = true)
 |    |    |    |-- ta: string (nullable = true)
 |-- vs_explod: struct (nullable = true)
 |    |-- a: boolean (nullable = true)
 |    |-- is: string (nullable = true)
 |    |-- p: string (nullable = true)
 |    |-- px: double (nullable = true)
 |    |-- py: double (nullable = true)
 |    |-- sv: string (nullable = true)
 |    |-- t: string (nullable = true)
 |    |

In [133]:
df_prev_chegada = df_explode.select(
    col("ps_explod.cp").alias("ID_PARADA"), 
    col("ps_explod.np").alias("nome_parada"),
    col("ps_explod.px").alias("latitude_loc"), 
    col("ps_explod.py").alias("longtitude"),
    col("vs_explod.a").alias("VEICULO_ACESSIVEL"),
    col("vs_explod.is").alias("TIMESTAMP"),
    col("vs_explod.p").alias("PREFIXO_VEICULO"),
    col("vs_explod.px").alias("LATITUDE_VEICULO"),
    col("vs_explod.py").alias("LONGITUDE_VEICULO"),
    col("vs_explod.t").alias("HORARIO_PREVISTO_CHEADA")    
)

In [134]:
silver_path_posicao_veiculo = 's3a://silver/previsao_chegada'
df_prev_chegada.write.format('delta')\
    .mode('overwrite').option("overwriteSchema", "true").save(silver_path_posicao_veiculo)

In [77]:

df_explode_ps.select("ps_explod.cp","ps_explod.np", "ps_explod.px", "ps_explod.py").show()

+---------+----+----------+----------+
|       cp|  np|        px|        py|
+---------+----+----------+----------+
|760012111|null|-46.435489|-23.495062|
|760002802|null|-46.443393|-23.509101|
|600005464|null|-46.527918|-23.528948|
|350002423|    |-46.387067|-23.508035|
|350002420|    |-46.388636|-23.506043|
|350002425|    | -46.38536|-23.510288|
|350002427|    |-46.383354|-23.511908|
|350002451|    |-46.397659|-23.501721|
|350002410|    |-46.400176|-23.498273|
|350002416|    |-46.393306|-23.500041|
|  3511854|    |-46.393636|-23.500189|
|860012107|    |-46.414106|-23.494477|
|860012108|    |-46.420331|-23.493976|
|860012106|    |-46.411507| -23.49483|
|860012105|    |-46.408131|-23.494799|
|760012110|    |-46.431702|-23.495227|
|860012124|    |-46.422071|-23.494154|
|860012109|    |-46.426964|-23.494623|
|760002790|    |-46.443165|-23.494625|
|760002793|    |-46.442241|-23.497567|
+---------+----+----------+----------+
only showing top 20 rows



In [103]:
silver_path_previsao_veiculo = 's3a://silver/previsao_chegada'

In [135]:
df_previsao_veiculo = spark.read.format('delta') \
  .load(silver_path_previsao_veiculo)

In [136]:
df_gold_diff = df_previsao_veiculo.filter("PREFIXO_VEICULO=11523").distinct().sort(col("HORARIO_PREVISTO_CHEADA"))

In [118]:
df_gold_diff.columns

['ID_PARADA',
 'nome_parada',
 'latitude_loc',
 'longtitude',
 'VEICULO_ACESSIVEL',
 'TIMESTAMP',
 'PREFIXO_VEICULO',
 'LONGITUDE_VEICULO',
 'LATITUDE_VEICULO',
 'HORARIO_PREVISTO_CHEADA']

In [124]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Converter graus para radianos
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Diferenças entre os pontos
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distância
    distance = R * c
    return distance

calc_km_long_lat = udf(lambda lat1, lon1, lat2, lon2: haversine(lat1, lon1, lat2, lon2))

In [137]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, radians, sin, cos, atan2, sqrt
from pyspark.sql.types import DoubleType
R = 6371.0

# Converter as colunas de latitude e longitude para radianos
df = df_gold_diff.withColumn("lat1_rad", radians(col("latitude_loc"))) \
       .withColumn("lon1_rad", radians(col("longtitude"))) \
       .withColumn("lat2_rad", radians(col("LATITUDE_VEICULO"))) \
       .withColumn("lon2_rad", radians(col("LONGITUDE_VEICULO")))

# Calcular a diferença entre as coordenadas
df = df.withColumn("dlat", col("lat2_rad") - col("lat1_rad")) \
       .withColumn("dlon", col("lon2_rad") - col("lon1_rad"))

# Aplicar a fórmula de Haversine
df = df.withColumn("a", sin(col("dlat") / 2)**2 + cos(col("lat1_rad")) * cos(col("lat2_rad")) * sin(col("dlon") / 2)**2)

df = df.withColumn("c", 2 * atan2(sqrt(col("a")), sqrt(1 - col("a"))))

# Calcular a distância final
df = df.withColumn("distancia_km", col("c") * R)

# Selecionar as colunas de interesse
df_final = df.select("latitude_loc", "longtitude", "LATITUDE_VEICULO", "LONGITUDE_VEICULO", "distancia_km")


In [138]:
df_final.show()

+------------+----------+-------------------+-------------------+------------------+
|latitude_loc|longtitude|   LATITUDE_VEICULO|  LONGITUDE_VEICULO|      distancia_km|
+------------+----------+-------------------+-------------------+------------------+
|  -46.653938|-23.523084|-46.648288333333326|-23.526863333333335|0.6912804698332657|
|  -46.658771|-23.522696|-46.648288333333326|-23.526863333333335|1.2082377393574435|
|  -46.664001|-23.521879|-46.648288333333326|-23.526863333333335| 1.788102844213461|
|  -46.673287|-23.479126|-46.648288333333326|-23.526863333333335| 4.582447213466181|
+------------+----------+-------------------+-------------------+------------------+



In [126]:
df_gold_diff.withColumn("diff_latitude", calc_km_long_lat(
    col("latitude_loc").cast('decimal'),
    col("longtitude").cast('decimal'),
    col("LATITUDE_VEICULO").cast('decimal'),
    col("LONGITUDE_VEICULO").cast('decimal'))).show()

+---------+--------------------+------------+----------+-----------------+---------+---------------+-------------------+-------------------+-----------------------+------------------+
|ID_PARADA|         nome_parada|latitude_loc|longtitude|VEICULO_ACESSIVEL|TIMESTAMP|PREFIXO_VEICULO|  LONGITUDE_VEICULO|   LATITUDE_VEICULO|HORARIO_PREVISTO_CHEADA|     diff_latitude|
+---------+--------------------+------------+----------+-----------------+---------+---------------+-------------------+-------------------+-----------------------+------------------+
|700016473|    ANHANGUERA - C/B|  -46.653938|-23.523084|             true|     null|          11523|-46.648288333333326|-23.526863333333335|                  20:09| 3272.209692597927|
|700013950|CAMISA VERDE E BR...|  -46.658771|-23.522696|             true|     null|          11523|-46.648288333333326|-23.526863333333335|                  20:10| 3272.209692597927|
| 60016465|QUIRINO DOS SANTO...|  -46.664001|-23.521879|             true|     n

In [96]:
silver_path_posicao_veiculo = 's3a://silver/posicao_veiculo'

In [113]:
df_posicao_veiculo = spark.read.format('delta') \
  .load(silver_path_posicao_veiculo)

In [114]:
df_posicao_veiculo.filter("ID_VEICULO=11523").distinct().show()

+--------------+--------+----------------------+---------------------+-----------------+---------------+-----------------+----------+-------------------+-------------------+
|LETREIRO_LINHA|ID_LINHA|LETREIRO_DESTINO_LINHA|LETREIRO_ORIGEM_LINHA|QTD_VEICULO_LINHA|SENTIDO_O_LINHA|ACESSIVEL_VEICULO|ID_VEICULO|  LONGITUDE_VEICULO|   LATITUDE_VEICULO|
+--------------+--------+----------------------+---------------------+-----------------+---------------+-----------------+----------+-------------------+-------------------+
|       9500-10|     461|       PÇA. DO CORREIO|   TERM. CACHOEIRINHA|                4|              1|             true|     11523|         -46.674865|         -23.499823|
|       9500-10|     461|       PÇA. DO CORREIO|   TERM. CACHOEIRINHA|                5|              1|             true|     11523|         -46.667728|        -23.4743105|
|       9500-10|     461|       PÇA. DO CORREIO|   TERM. CACHOEIRINHA|                4|              1|             true|     115